In [1]:
%matplotlib inline
import numpy as np
import nbimporter
from _helper_functions import find_nearest_sorted, odeintz
from _mbes import mbes, mbes_soc, mbes_soc_func, mbes_simulate_losses

Importing Jupyter notebook from _helper_functions.ipynb
Importing Jupyter notebook from _mbes.ipynb


In [2]:
def solve_mbes(mbes_func, init, pdf, tlistpump, tlistdecay, *args):
    #mbes_func: MBE functions to solve (defined in _mbes)
    #init: inital state
    #pdf: pdf for spins, necessary for normalizing the solutions
    #tlistpump: timelist for which the pump tone is turned on
    #tlistdecay: timelist for which the decay is computed (keep None if you don't want any decay)
    #*args: function arguments to pass to the MBE functions 
      
    solpump, infodict = odeintz(mbes_func, init, tlistpump, args=(*args, ), full_output=True)
    
    if tlistdecay is not None:
        init2 = solpump[-1,:].copy()
        #make sure that the last six arguments correspond to the one defined in the mbes function in _mbes
        soldecay = odeintz(mbes, init2, tlistdecay, args=(0, *args[-6:]))
        
        solpump[:,2::2] = solpump[:,2::2]*pdf
        solpump[:,1::2] = solpump[:,1::2]*pdf
        soldecay[:,2::2] = soldecay[:,2::2]*pdf
        soldecay[:,1::2] = soldecay[:,1::2]*pdf
        return solpump, soldecay, infodict
    else:
        solpump[:,2::2] = solpump[:,2::2]*pdf
        solpump[:,1::2] = solpump[:,1::2]*pdf
        return solpump, infodict